# Script to create composites of SST gradient based on ENSO phase

In [155]:
import numpy as np
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
import xarray as xr

from grad_funcs import get_gs_coords_alltime,get_total_gradient

In [156]:
## months to perform analysis, for full year set analysis_months = range(1,13)

analysis_months = [2]


dslink = '/Users/noahrosenberg/Downloads/era5_shf_1deg.nc'#era5_sst_1940_2022_1deg.nc'

ds = xr.load_dataset(f'{sstlink}',engine='netcdf4')
ds = ds.where(ds.time.dt.month.isin(analysis_months))

/Users/noahrosenberg/miniconda3/lib/python3.11/site-packages/xarray/core/accessor_dt.py:72: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  values_as_series = pd.Series(values.ravel(), copy=False)


In [159]:
ds

<xarray.Dataset>
Dimensions:    (time: 30200, latitude: 37, longitude: 249)
Coordinates:
  * longitude  (longitude) float32 -76.0 -75.75 -75.5 ... -14.5 -14.25 -14.0
  * latitude   (latitude) float32 28.0 27.75 27.5 27.25 ... 19.5 19.25 19.0
  * time       (time) datetime64[ns] 1940-01-01 ... 2023-02-28T18:00:00
Data variables:
    sst        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    sshf       (time, latitude, longitude) float32 nan nan ... -1.418e+05
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-03 13:43:29 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [158]:
sst = ds.sshf

AttributeError: 'Dataset' object has no attribute 'shf'

In [ ]:
## get dates from Nino script and dataarray into same format

nino_dates = np.load('ninodates.npy')

nino_months = nino_dates.astype('datetime64[M]')

year0 = sst.time.dt.year.min()
year0 = int(year0)

nino_indices = nino_months.astype('int')+(1970-year0)*12

sst_month_indices = (sst.time.dt.year-1940)*12 + sst.time.dt.month

sst_nino = sst.where(sst_month_indices.isin(nino_indices),drop=True)
sst_nino = sst_nino

In [ ]:
fig,ax = plt.subplots(figsize=(10,10),subplot_kw={'projection':ccrs.PlateCarree()})
im = ax.pcolormesh(sst_nino.longitude,sst_nino.latitude,sst_nino.mean('time'),transform=ccrs.PlateCarree())
#im2 = ax.contour(sst.lon,sst.lat,tot_grad.mean('time'),levels=(1,288),c='r')

#ax.set_extent((-80,-40,20,75))
plt.colorbar(im,ax=ax)
ax.coastlines()

In [ ]:
nina_dates = np.load('ninadates.npy')

nina_months = nina_dates.astype('datetime64[M]')

year0 = sst.time.dt.year.min()
year0 = int(year0)

nina_indices = nina_months.astype('int')+(1970-year0)*12

sst_month_indices = (sst.time.dt.year-1940)*12 + sst.time.dt.month

sst_nina = sst.where(sst_month_indices.isin(nina_indices),drop=True)

In [ ]:
from matplotlib import colors

# plot difference in gradients between 2 composites

nina_grad = get_total_gradient(sst_nina,1,latname='latitude',lonname='longitude').mean('time')
nino_grad = get_total_gradient(sst_nino,1,latname='latitude',lonname='longitude').mean('time')

fig,ax = plt.subplots(figsize=(5,5),subplot_kw={'projection':ccrs.PlateCarree()})
im = ax.pcolormesh(sst_nina.longitude,sst_nina.latitude,
                   nino_grad-nina_grad,
                   transform=ccrs.PlateCarree(),
                   vmin=-0.005,vmax=0.005,cmap="RdBu_r")#,norm=colors.CenteredNorm(0,1),cmap='RdBu_r')
im2 = ax.contour(sst.longitude,sst.latitude,sst.mean('time'),levels=20,colors='k',linewidths=0.5)

#im2 = ax.contour(sst.lon,sst.lat,tot_grad.mean('time'),levels=(1,288),c='r')

#ax.set_extent((-80,-40,20,75))
plt.colorbar(im,ax=ax)
ax.coastlines()